In [ ]:
import numpy as np
# import pandas as pd
import scanpy as sc
import seaborn as sb
import matplotlib.pyplot as plt
# from matplotlib import rcParams
%matplotlib inline
# import seaborn as sns
import scipy.sparse
# import cmasher as cmr
# from natsort import natsorted
import sys 
import os
import joblib
# from imblearn.over_sampling import SMOTE
# from collections import Counter
# import sc_module as sm
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
# from collections import OrderedDict
# import decoupler as dc

sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80)

In [ ]:
sys.path.append(os.path.abspath(r"C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 2\\jl_modules"))

In [ ]:
clustered_file = "C:\\Users\\bence\\Projects\\BIO446\\McKinnon-Rosati-Laboratory\\Project 1\\granulomas_final.h5ad"

In [ ]:
# clustered data
adata = sc.read_h5ad(clustered_file)
adata.uns['log1p']["base"] = None # bug fix

print(adata.obs['sample'].value_counts())
print()
print('X matrix is sparse:', scipy.sparse.issparse(adata.X))
print('X size =', adata.X.shape)

In [ ]:
adata

In [ ]:
cluster_type = 'my_clust_1'

In [ ]:
adata.obs[cluster_type]

In [ ]:
annotation_dict= {
'9': 'CAP1',
'24': 'CAP2',
'9b': 'VEC',
'27': 'LEC',
    
'17': 'Ciliated',
'15': 'Secretory',
'22': 'AT1',
'6': 'AT2',
'12': 'AT2-t1',
'19': 'AT2-t2',
    
'18': 'AF1',
'14': 'AF2',
'25': 'Pericyte',
    
'20': 'Mesothelial',
    
'3': 'B1',
'3b': 'B2',
    
'0': 'Th1',
'8': 'T_naive',
'11': 'T_ex',
'77': 'Treg',
    
'11b': 'NK',
    
'4a': 'AM',
'4': 'M-t1',
'10': 'M-lc',
'7': 'M-t2',
'7b': 'M-C1q',
'7c': 'iMon',
    
'23': 'pDC',
'13': 'DC',
'5b': 'N1',
'5': 'N2',
}

In [ ]:
# Add cell type column based on annotation
adata.obs['cell_type_edit'] = [annotation_dict[clust] for clust in adata.obs[cluster_type]]

# Error checking
dict_list = list(annotation_dict.keys())
adata_list = list(adata.obs['my_clust_1'].unique())
print('Keys in dictionary not in adata:', [item for item in dict_list if item not in adata_list] )
print('Keys in adata not in dictionary:', [item for item in adata_list if item not in dict_list] )

In [ ]:
adata.obs['cell_type_edit']

In [ ]:
# collect unique cells
unique_celltype_sub = adata.obs['cell_type_edit'].unique()
print(unique_celltype_sub)

# count num unique cells
num_unique_celltype_sub = adata.obs['cell_type_edit'].nunique()
print(f"Number of unique sub cell types: {num_unique_celltype_sub}")

In [ ]:
# map cell types to integers
replacement_dict = {
    'AT2': 0,
    'B1': 1,
    'M-t1': 2,
    'DC': 3,
    'Th1': 4,
    'M-t2': 5,
    'Secretory': 6,
    'AM': 7,
    'N1': 8,
    'M-C1q': 9,
    'AT2-t2': 10,
    'AF2': 11,
    'VEC': 12,
    'CAP1': 13,
    'N2': 14,
    'AT2-t1': 15,
    'Pericyte': 16,
    'pDC': 17,
    'Ciliated': 18,
    'NK': 19,
    'AT1': 20,
    'T_naive': 21,
    'Treg': 22,
    'M-lc': 23,
    'Mesothelial': 24,
    'T_ex': 25,
    'CAP2': 26,
    'LEC': 27,
    'iMon': 28,
    'B2': 29
}

In [ ]:
adata.obs['cell_type_edit2'] = adata.obs['cell_type_edit']
adata.obs['celltype'] = adata.obs['cell_type_edit'].replace(replacement_dict)

In [ ]:
adata.obs['celltype']

In [ ]:
print("Unique values in 'celltype' column after replacement:", adata.obs['celltype'].unique())

In [ ]:
# number of cells in each cell type
cell_type_counts = adata.obs['cell_type_edit2'].value_counts()

print("Number of cells in each cell type:")
print(cell_type_counts)

In [ ]:
# cell distribution with cell types (hard to see)
y = adata.obs['cell_type_edit2']
plt.figure(figsize=(35, 18))
plt.title('Cell Distribution with Cell Types')
y.value_counts().plot(kind='bar', rot=0)
plt.show()

In [ ]:
# cell distribution with int mappings (easy to see)
y = adata.obs['celltype']
plt.figure(figsize=(14, 8))
plt.title('Cell Distribution with Integer Mappings')
y.value_counts().plot(kind='bar', rot=0)
plt.show()

In [ ]:
# Extract gene expression values (X) and cell types (y) from the adata object
# If adata.X is already a numpy array, just use it directly

X = adata.X  # No need for .toarray() if it's already a numpy array
y = adata.obs['celltype'].values  # Extract the cell types as a numpy array

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size = 0.2, random_state=42)
train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size = 0.25, random_state=42)

train_features = np.array(train_features)
test_features = np.array(test_features)
val_features = np.array(val_features)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)

# Print the shape of the training and testing sets
print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

In [ ]:
print(y)
print(len(y))

In [ ]:
print(X)
print(len(X))

In [ ]:
adata.obs['celltype']

In [ ]:
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=train_labels
)

In [ ]:
print(train_labels)
print(sample_weights)
print(len(sample_weights))

In [ ]:
# https://xgboost.readthedocs.io/en/latest/python/python_api.html

XGB_decisionTree = xgb.XGBClassifier(
    n_estimators=500,              # More trees for better performance
    learning_rate=0.05,            # Slower learning rate
    max_depth=8,                   # Moderate depth for balance
    colsample_bytree=0.8,          # Feature sampling for generalization
    subsample=0.8,                 # Row sampling for generalization
    gamma=0.1,                     # Loss reduction threshold
    objective='multi:softmax',     # Multiclass classification
    early_stopping_rounds=25
)

XGB_decisionTree = XGB_decisionTree.fit(X=train_features, y=train_labels, eval_set=[(val_features, val_labels)], sample_weight=sample_weights)

In [ ]:
XGB_decisionTree_pred = XGB_decisionTree.predict(test_features)

In [ ]:
def evaluate_model(y_true, y_pred, model_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"------ {model_name} ------")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

In [ ]:
evaluate_model(test_labels, XGB_decisionTree_pred, "XGBoost Decision Tree")

In [ ]:
def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7.7, 6))
    sb.heatmap(cm, annot=False, cmap='Blues', cbar=True,
                xticklabels=np.unique(y), yticklabels=np.unique(y))
    plt.title(f'Confusion Matrix - {model_name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

In [ ]:
plot_confusion_matrix(test_labels, XGB_decisionTree_pred, "XGBoost Decision Tree")

In [ ]:
joblib.dump(XGB_decisionTree, 'XGBmodel.pkl')